# Uvozim knjižice

In [1]:
import spacy #

import warnings
warnings.filterwarnings('ignore')
from spacy import displacy
import nltk
from tqdm.notebook import tqdm
import networkx as nx
import re #regular expression

# Priprava besedila

In [2]:
#1 Naložimo besedilo
book=open("GOT.txt",encoding="utf8").read()

#2 Čiščenje besedila
book=re.sub(r"“","",book)
book=re.sub(r"”","",book)

#3 Diskretizacija besedila
stavki=nltk.sent_tokenize(book)
print(f"Stevilo stavkov v besedilu je {len(stavki)}")

Stevilo stavkov v besedilu je 89040


# NER (Named Enteties Recognition): Spacy

In [10]:
nlp=spacy.load("en_core_web_sm")

<img src="https://user-images.githubusercontent.com/13643239/55229632-dbff9480-521d-11e9-8499-efb2a9c948db.png" alt="Slika_1" width="900"/>

In [50]:
liki={} #Prazen dictionary za like in lege, kje se pojavljajo.

N=int(0.2*len(stavki)) #Definiram kolikšen delež oz. število stavkov uporabi v analizi

Sedja lahko uporabimo spacy, da prepoznamo za kakšno vrsto besede gre oz. ali ima posamezna beseda določen pomen kot je to recimo ime osebe. V spodnji zanki gremo skozi prvih 10 stavkov in preverimo kakšne so entitete v besedilu.

In [51]:
for s in tqdm(range(100),total=100):
    if len(stavki[s])>100:
        doc=nlp(stavki[s])
        try:
            displacy.render(doc, style="ent", jupyter=True)
        except:
            pass

  0%|          | 0/100 [00:00<?, ?it/s]

In [52]:
for s in tqdm(range(N),total=N):
    doc=nlp(stavki[s])
    for ent in doc.ents:
        if ent.label_=="PERSON":
            oseba=ent.text
            if oseba not in liki:
                liki[oseba]=[]
                #print(s,oseba, ent.start_char, ent.end_char, ent.label_)
            liki[oseba].append(s) # Za osebo si zapomnimo v katerem zaporednem stavku se je pojavila.

  0%|          | 0/17808 [00:00<?, ?it/s]

In [53]:
for oseba in list(liki.keys())[:10]:
    print(oseba,liki[oseba])

Gared [0, 11, 20, 25, 49, 54, 58, 111, 114, 126, 138, 195, 212, 217]
Waymar Royce [3, 23, 47, 160, 173, 236, 301, 345, 373, 5980, 7885]
grey-eyed [48]
Waymar [51, 112, 126, 129, 133, 177, 213, 249, 253, 285, 312, 322, 328, 335, 342]
Mallisters [73]
m’lord [108, 178, 202, 4349, 4815, 7894, 8884, 9128, 9138, 9144, 10847, 10865, 11131, 11923, 12645, 12660, 12669, 14111, 14360, 14361, 14362, 14367, 14411, 14417, 14418, 14970, 15796, 15810, 16222, 17179, 17238]
Maester Aemon [135, 369, 5710, 5794, 6577, 6581, 6583, 6649, 8396, 8571, 13851, 13879, 13906, 13912, 13921, 13929, 13938, 13951, 13966, 13980, 13993, 15969, 15982, 16084, 16103, 17139, 17276, 17334, 17356, 17588]
Will [144, 170]
Royce [169]
wolf [174, 704, 1608, 1721, 2297, 2301, 2532, 2536, 2538, 2540, 2543, 2783, 3834, 4006, 4176, 4263, 4269, 4534, 4649, 4683, 4727, 5279, 7102, 7843, 10668, 16226, 16698, 17658, 17766, 17781]


# Izgradnmja sociograma

## Poselimo mrežo z vozlišči

In [54]:
#Ustvariom prazno mrežo.
G=nx.Graph()

#Mrežo poselimo z vozlišči.
for node in liki:
    G.add_node(node)

nodes=[node for node in G]

In [55]:
TH_LINK=3
def check_dist(TH_LINK,x,y):
    '''
    x -> seznam lokacij za lik 1
    y -> seznam lokacij za lik 2
    '''
    xx=np.array(x)
    yy=np.array(y)
    edge=None
    z=[min(abs(s-xx)) for s in yy]
    edge=len([zz for zz in z if zz<TH_LINK])
    return edge

def create_sociogram(liki,TH,TH_LINK):
    #Ustvariom prazno mrežo.
    G=nx.Graph()

    #Mrežo poselimo z vozlišči.
    for node in liki:
        G.add_node(node)

    for i,ix in tqdm(enumerate(nodes),total=len(nodes)):
        for iy in nodes[i+1:]: #Zgornja trikotna matrika
            wij=check_dist(TH_LINK,liki[ix],liki[iy])
            if wij>=TH:
                G.add_edge(ix,iy,weight=wij)
    return G

## Ustvarimo povezave

Poveza do progrma vizualizacijo in analizo mrež <a href="https://gephi.org/"> GEPHI </a>

In [56]:
for TH in range(0,10):
    G=create_sociogram(liki,TH,TH_LINK)
    print(f"TH = {TH}, <k> = {np.mean([G.degree(node) for node in G]):.2f}")

  0%|          | 0/541 [00:00<?, ?it/s]

TH = 0, <k> = 540.00


  0%|          | 0/541 [00:00<?, ?it/s]

TH = 1, <k> = 8.12


  0%|          | 0/541 [00:00<?, ?it/s]

...

TH = 7, <k> = 0.25


  0%|          | 0/541 [00:00<?, ?it/s]

TH = 8, <k> = 0.20


  0%|          | 0/541 [00:00<?, ?it/s]

TH = 9, <k> = 0.18


In [58]:
G=create_sociogram(liki,3,TH_LINK)

node_to_remove=[]

for node in G:
    if G.degree(node)<1:
        node_to_remove.append(node)

for node in node_to_remove:
    G.remove_node(node)

nx.write_graphml(G,"sociogram_GOT.graphml")

  0%|          | 0/541 [00:00<?, ?it/s]

# Sentimentalna vrednost stavka

In [64]:
from spacytextblob.spacytextblob import SpacyTextBlob
#nlp.add_pipe('spacytextblob')
print(nlp.pipe_names)

['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner', 'spacytextblob']


In [69]:
for s in tqdm(range(10),total=10):
    doc=nlp(stavki[s])
    sentiment=doc._.blob.polarity
    if abs(sentiment)>0.7:
        print(f"{doc} | polarity = {sentiment:.2f}")

  0%|          | 0/10 [00:00<?, ?it/s]

In [73]:
text='Janez1: "Asdlčfkjasčdlkf jčadsfg"\n.P1eter: "lkasjdhflkasdjhfasf"\nMarija: "lksadjflkasjf"'

In [71]:
import re

In [74]:
text=re.sub(r"[A-Z]+[a-z0-9]+:\s",'',text)
print(text)

"Asdlčfkjasčdlkf jčadsfg"
."lkasjdhflkasdjhfasf"
"lksadjflkasjf"
